# Start Simulation

In [1]:
import random
import numpy as np
import os
import time
from PIL import Image

import torch
torch.__version__
device = torch.device("cuda")

In [2]:
import getpass
user = getpass.getuser()
print(user)

yizhou


In [3]:
usd_path = f"omniverse://localhost/Users/{user}/uva_sac.usd"

In [4]:
from omni.isaac.kit import SimulationApp    
simulation_app = SimulationApp({"headless": True, "open_usd": usd_path,  "livesync_usd": usd_path}) 

Starting kit application with the fillowing args:  ['/home/yizhou/.local/share/ov/pkg/isaac_sim-2022.1.1/exts/omni.isaac.kit/omni/isaac/kit/simulation_app.py', '/home/yizhou/.local/share/ov/pkg/isaac_sim-2022.1.1/apps/omni.isaac.sim.python.kit', '--/app/tokens/exe-path=/home/yizhou/.local/share/ov/pkg/isaac_sim-2022.1.1/kit', '--/persistent/app/viewport/displayOptions=3094', '--/rtx/materialDb/syncLoads=True', '--/rtx/hydra/materialSyncLoads=True--/omni.kit.plugin/syncUsdLoads=True', '--/app/renderer/resolution/width=1280', '--/app/renderer/resolution/height=720', '--/app/window/width=1440', '--/app/window/height=900', '--/renderer/multiGpu/enabled=True', '--ext-folder', '/home/yizhou/.local/share/ov/pkg/isaac_sim-2022.1.1/exts', '--ext-folder', '/home/yizhou/.local/share/ov/pkg/isaac_sim-2022.1.1/apps', '--/physics/cudaDevice=0', '--portable', '--no-window']
Passing the following args to the base kit application:  ['-f', '/home/yizhou/.local/share/jupyter/runtime/kernel-2fed25f9-a2d9-

OmniAssetFileFormat


Warp initialized:
   Version: 0.2.2
   CUDA device: NVIDIA GeForce RTX 3090
   Kernel cache: /home/yizhou/.cache/warp/0.2.2
[1.788s] [ext: omni.kit.menu.create-1.0.2] startup
[1.789s] [ext: omni.mdl-0.1.0] startup
[1.814s] [ext: omni.kit.window.file_importer-1.0.4] startup
[1.815s] [ext: omni.kit.window.file_exporter-1.0.4] startup
[1.817s] [ext: omni.kit.material.library-1.3.10] startup
[1.823s] [ext: omni.kit.window.drop_support-1.0.0] startup
[1.824s] [ext: omni.kit.window.file-1.3.16] startup
[1.826s] [ext: omni.kit.context_menu-1.3.9] startup
[1.830s] [ext: omni.kit.window.property-1.6.3] startup
[1.831s] [ext: omni.kit.window.content_browser-2.4.28] startup
[1.841s] [ext: omni.kit.widget.stage-2.6.15] startup
[1.845s] [ext: omni.isaac.version-1.0.0] startup
[1.846s] [ext: omni.kit.property.usd-3.14.9] startup
[1.879s] [ext: omni.kit.viewport.legacy_gizmos-1.0.0] startup
[1.883s] [ext: omni.hydra.rtx-0.1.0] startup
[1.893s] [ext: omni.renderer-rtx-0.0.0] startup
[1.894s] [ext: omn

In [5]:
# set log level
import logging
import carb

logging.getLogger("omni.hydra").setLevel(logging.ERROR)
logging.getLogger("omni.isaac.urdf").setLevel(logging.ERROR)
logging.getLogger("omni.physx.plugin").setLevel(logging.ERROR)

logging.getLogger("omni.isaac.synthetic_utils").setLevel(logging.ERROR)
logging.getLogger("omni.isaac.synthetic_utils.syntheticdata").setLevel(logging.ERROR)
logging.getLogger("omni.hydra.scene_delegate.plugin").setLevel(logging.ERROR)


l = carb.logging.LEVEL_ERROR
carb.settings.get_settings().set("/log/level", l)
carb.settings.get_settings().set("/log/fileLogLevel", l)
carb.settings.get_settings().set("/log/outputStreamLevel", l)

# # This logged error is printed as it should
# carb.log_error("ERROR")
# # This warning is printed but should not
# carb.log_warn("WARNING")

# Config

In [6]:
from task.config import DATA_PATH, FEATURE_PATH
task_type = "Table"
side_choice = "Border"
base_asset_id = 0
load_nucleus = True

In [7]:
pause

NameError: name 'pause' is not defined

# Init Env

In [ ]:
from uva_env import UvaEnv
env = UvaEnv()

In [ ]:
from task.utils import add_scene_default
add_scene_default()

In [ ]:
print(list(env.stage.TraverseAll()))

In [ ]:
env.clean()
env.world.step(render=True)

# Scene

In [ ]:
from task.scene import ArrangeScene
scene = ArrangeScene(task_type, side_choice, base_asset_id = 0, traj_id = 0, load_nucleus = load_nucleus)
env.scene = scene

In [ ]:
# add base
scene.add_base_asset()

In [ ]:
# add room
# scene.add_room()

In [ ]:
env.world.step(render=True)

# Reward

In [ ]:
from uv.reward import Rewarder

In [ ]:
rewarder = Rewarder(env.world)
env.rewarder = rewarder

# Render

In [ ]:
from render.helper import RenderHelper
render = RenderHelper(task_type, side_choice)

render.add_task_cameras()
render.set_cameras()

In [ ]:
pause

#  Learning

In [ ]:
# from learning.network.resnet import ResNetFeatureExtractor

# feature extraction
from learning.utils import extract_image_clip_feature_and_save, obtain_action_from_trainer
from transformers import CLIPProcessor, CLIPModel

feature_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
feature_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

In [ ]:
# replay buffer
import json
from learning.replay_buffer import ReplayBuffer
from learning.config import *

buffer = ReplayBuffer(max_size=2000)

In [ ]:
# trainer
from learning.network.sac import *

policy = Policy()

qf1 = QFunction()
qf2 = QFunction()
target_qf1 = QFunction()
target_qf2 = QFunction()

from learning.sac_trainer import SACTrainer

trainer = SACTrainer(policy, qf1, qf2, target_qf1, target_qf2, 
     use_automatic_entropy_tuning = False, 
     policy_lr=1e-3, 
     qf_lr=1e-3,
     target_update_period = 5)

# Trajectory

In [ ]:
use_network = True
debug = True

In [ ]:
total_traj = 0
total_step = 0

---------------------

In [ ]:
# traj config
for traj_id in range(200):
    total_traj += 1
    
    base_asset_id = 0
    env.scene.base_asset_id = base_asset_id
    env.scene.traj_id = traj_id
    image_folder = os.path.join(DATA_PATH, task_type, side_choice, str(traj_id))

    # base
    # scene.add_base_asset()
    env.world.step(render = True)

    # get images
    env.world.render()
    images = render.get_images()
    render.save_rgb(images[0]['rgb'], image_folder, "0")
    
    ## extract feature
    
#     extract_image_feature_and_save(images[0]['rgb'][:,:,:3], 
#         feature_extractor, os.path.join(image_folder, str(0) + ".pt"))
    extract_image_clip_feature_and_save(f"{image_folder}/{0}.png", feature_model, feature_processor, 
        f"{image_folder}/{0}.pt",)
    

    # trajectory
    for step in range(5):
        total_step += 1
        
        # sample an object
        env.add_scene_obj(mode = "random")
        
        # TODO: get action from sampling
        if not use_network or total_traj < 10 or np.random.rand() < 0.2:
            x, y = np.tanh(np.random.randn()), np.tanh(np.random.randn())
        else:
            image_feature_file = f"{image_folder}/{step}.pt"
            
            object_info = env.scene.objects[-1]
            object_type = object_info["type"]
            obj_name = object_info["name"][:-4]
            object_feature_file = os.path.join(FEATURE_PATH, object_type, obj_name + ".pt")
            x, y = obtain_action_from_trainer(image_feature_file, object_feature_file, trainer)
        
        # load the object into the scene
        env.put_last_object((x, y)) 
        env.world.step(render=True)
        
        # register the object to the world for physics update
        env.register_last_object()
        env.world.step(render=True)

        # get images
        env.world.render()
        images = render.get_images()
        render.save_rgb(images[0]['rgb'], image_folder, str(step + 1))

        ## calculate reward
        env.calculate_last_reward(simulation_step = 30)
        
        ## extract feature
#         extract_image_feature_and_save(images[0]['rgb'][:,:,:3], 
#             feature_extractor, os.path.join(image_folder, str(step + 1) + ".pt"))
        extract_image_clip_feature_and_save(f"{image_folder}/{step + 1}.png", feature_model, feature_processor, 
            f"{image_folder}/{step + 1}.pt",)
    

        ## reset
        env.world.reset()
        env.world.step(render=True)
        
        ## trainer nework
        if use_network and total_step % UPDATE_TRAINER_STEPS == 0 and total_traj > 5:
            batch = buffer.sample_batch(batch_size = BATCH_SIZE)
            trainer.update(batch)
            
            if debug:
                rewards = batch['rewards'].to(device)
                terminals = batch['terminals'].to(device)
                obs = batch['observations'].to(device)
                actions = batch['actions'].to(device)
                next_obs = batch['next_observations'].to(device)
                obj_features = batch['object_features'].to(device)
                
                dist = trainer.policy(obs, obj_features)
                pred = trainer.qf1(obs, obj_features, actions)
                print(f"debug {total_traj}/{total_step}", #"\n dist: ", dist.mean.flatten().tolist(), dist.stddev.flatten().tolist(),
                      "\n pred:", pred.flatten().tolist(),
                      "\n rewards: ", rewards.flatten().tolist())


    # Record
    record = env.scene.get_scene_data()
    env.scene.save_scene_data()
    # print("record: ", record)
    
    # Add record to buffer
    buffer.add_scene_sample(record)

    # Reset (env clean)
    env.clean(clean_all = False)
    env.step(render = True)

# Debug

In [ ]:
# get images
env.world.render()
env.world.render()
images = render.get_images()

Image.fromarray(images[0]['rgb'], "RGBA").resize((300, 300))

##  Debug Training

In [8]:
import json
from learning.replay_buffer import ReplayBuffer

buffer = ReplayBuffer(max_size=1000)

for i in range(100):
    replay = json.load(open(f"{DATA_PATH}/{task_type}/{side_choice}/{i}/scene.json"))
    buffer.add_scene_sample(replay)

In [9]:
len(buffer.dataset)

500

In [10]:
# trainer
from learning.network.sac import *

policy = Policy()

qf1 = QFunction()
qf2 = QFunction()

target_qf1 = QFunction()
target_qf1.eval()

target_qf2 = QFunction()
target_qf2.eval()

from learning.sac_trainer import SACTrainer

trainer = SACTrainer(policy, qf1, qf2, target_qf1, target_qf2, 
     use_automatic_entropy_tuning = True, 
     policy_lr=1e-3, 
     qf_lr=1e-3,
     target_update_period = 5)

In [11]:
self = trainer

In [12]:
for _ in range(500):
    batch = buffer.sample_batch(batch_size=6)

    rewards = batch['rewards'].to(device)
    terminals = batch['terminals'].to(device)
    obs = batch['observations'].to(device)
    actions = batch['actions'].to(device)
    next_obs = batch['next_observations'].to(device)

    obj_features = batch['object_features'].to(device)

    # print("rewards", rewards)

    self.update(batch)
    
    if _ % 10 == 0:
        dist = self.policy(obs, obj_features)
        pred = self.qf1(obs, obj_features, actions)
        print(_, "\n dist   : ", [round(x, 3) for x in dist.mean.flatten().tolist()], 
                  [round(x, 3) for x in dist.stddev.flatten().tolist()],
              "\n pred   :", [round(x, 3) for x in pred.flatten().tolist()],
              "\n rewards: ", [round(x, 3) for x in rewards.flatten().tolist()])
    
#     # debug
#     pred = self.qf1(obs, obj_features, actions)
#     loss = self.qf_criterion(pred, rewards)
    
#     self.qf1_optimizer.zero_grad()
#     loss.backward()
#     self.qf1_optimizer.step()
    
#     if _ % 20 == 0:
#         print(_, "\n debug loss: ", loss.item(), "\n pred:", 
#               pred.flatten().tolist(), "\n rewards: ", rewards.flatten().tolist())

1.2] startup
[7.447s] [ext: omni.graph.window.core-1.23.4] startup
[7.454s] [ext: omni.graph.instancing-1.1.4] startup
[7.462s] [ext: omni.kit.menu.edit-1.0.6] startup
[7.463s] [ext: omni.isaac.isaac_sensor-1.0.2] startup
2022-12-24 19:30:03 [7,464ms] [Warning] [omni.physx.plugin] Deprecated: getSimulationEventStream is deprecated, please use getSimulationEventStreamV2
[7.475s] [ext: omni.graph.tutorials-1.1.2] startup
[7.493s] [ext: omni.graph.window.action-1.3.8] startup
[7.494s] [ext: omni.kit.widget.live-0.1.0] startup
[7.499s] [ext: omni.kit.widget.layers-1.5.17] startup
[7.512s] [ext: omni.graph.bundle.action-1.0.0] startup
[7.512s] [ext: omni.isaac.range_sensor-0.4.3] startup
[7.539s] [ext: omni.kit.property.layer-1.1.2] startup
[7.541s] [ext: omni.replicator.isaac-1.3.2] startup
[7.622s] [ext: omni.isaac.kit-0.2.1] startup
[7.622s] [ext: omni.graph.window.generic-1.3.8] startup
[7.624s] [ext: omni.isaac.utils-0.1.11] startup
[7.627s] [ext: omni.kit.property.audio-1.0.5] startup

200 
 dist   :  [0.182, 0.415, 0.101, 0.359, 0.194, 0.423, 0.167, 0.522, 0.138, 0.498, 0.07, 0.319] [0.635, 0.817, 0.725, 0.767, 0.632, 0.817, 0.578, 0.701, 0.608, 0.719, 0.76, 0.814] 
 pred   : [-0.012, 0.66, 1.006, 0.488, 1.299, 0.666] 
 rewards:  [-0.839, -0.839, 0.953, -0.839, -1.311, -0.839]
210 
 dist   :  [-0.068, 0.391, -0.077, 0.388, -0.08, 0.348, -0.058, 0.334, -0.063, 0.398, -0.056, 0.516] [0.729, 0.794, 0.736, 0.782, 0.766, 0.782, 0.774, 0.821, 0.697, 0.81, 0.615, 0.715] 
 pred   : [0.248, -0.124, 0.142, 0.008, -0.046, -0.149] 
 rewards:  [-0.839, -0.839, -0.839, -0.839, -0.839, -0.839]
220 
 dist   :  [-0.137, 0.298, -0.126, 0.328, -0.132, 0.337, -0.129, 0.313, -0.14, 0.391, -0.138, 0.33] [0.772, 0.835, 0.755, 0.816, 0.766, 0.795, 0.778, 0.809, 0.694, 0.765, 0.783, 0.792] 
 pred   : [0.272, 0.61, 0.412, 1.822, 1.336, 1.38] 
 rewards:  [-0.839, -0.839, -0.839, 0.801, 0.895, -0.839]
230 
 dist   :  [0.026, 0.28, 0.008, 0.277, 0.027, 0.301, -0.002, 0.259, 0.008, 0.308, 0.04, 

In [13]:
dist = self.policy(obs.to(device), obj_features.to(device))

In [14]:
dist.normal_mean, dist.stddev

(tensor([[0.1245, 0.3667],
         [0.0891, 0.3645],
         [0.1043, 0.3513],
         [0.0982, 0.3493],
         [0.0723, 0.3286],
         [0.0853, 0.3235]], device='cuda:0', grad_fn=<AddmmBackward0>),
 tensor([[0.5339, 0.4756],
         [0.5521, 0.4969],
         [0.5505, 0.5017],
         [0.5622, 0.5049],
         [0.5518, 0.5127],
         [0.5967, 0.5297]], device='cuda:0', grad_fn=<ExpBackward0>))

In [25]:
dist.sample(scaler = 0.2)

tensor([[ 0.1772,  0.3092],
        [-0.1434,  0.4958],
        [ 0.1856,  0.3790],
        [ 0.3934,  0.3446],
        [-0.0539,  0.2383],
        [ 0.0032,  0.2280]], device='cuda:0')

In [ ]:
self = dist

In [ ]:
scaler = 0.1

In [ ]:
self.normal_mean + scaler * self.normal_std * MultivariateDiagonalNormal(
    torch.zeros(self.normal_mean.size()).to(self.device),
    torch.ones(self.normal_std.size()).to(self.device)
).sample()

# distribution

In [ ]:
from learning.distributions import MultivariateDiagonalNormal

In [ ]:
dist = JustNormal(dist.mean, dist.stddev)

In [ ]:
dist.rsample_and_logprob()